In [1]:
import pandas as pd
import numpy as np

# Data Cleaning and Rough Draft for ACFT Calculaor 

## I took excel tables from here: https://companyleader.themilitaryleader.com/2019/11/07/acft-calculator-tracker/
## That are accurate to the US Army PDF that's more difficult to take data from here: https://www.goarmy.com/how-to-join/requirements/fitness

### Being new to Swift, I chose to write a rough draft version of the main calculations in Python. Knowing that Swift seems to favor funciotnal programming I went with dictionaries over Python objects. This also made sense with the end goal being to place the data in JSON (since JavaScript objects are pretty similar to Python dictionaries)

### I felt the author, "The Company Leader", of the Excel calculator was too faithful to the Army website's formatting. I'm taking his excel tables, turning them to pandas data frames, subsetting them as needed to make the JSON file for the actual app. There's also a function to do the primary logic of the iOS app that takes a dictionary as an argument and calculate an indivual's score. 

### There are three stages to this app: (1) basic iOS ACFT calculator for an individual, (2) add SQLite for tracking individual historic performance metrics and for unit leaders to quickly log results for their soldiers and recieve an Excel document with the results, and (3) create a feature to scan handwriting on DA 705s to instantly get ACFT scores.  This notebook is only relevant for stage one of this app. 

## converting excel tables to data frames to be cleaned for each event 

 ### copied and pasted score charts into my own excel file and creating data frames from there 

In [76]:
# the big adjustments are really just setting this up to be one bit list of dictionaries for a JSON document 
# I added gender, so that it'll be easy to filter later 

# male scoring charts
male_dl = pd.read_excel("scoreChartsACFT.xlsx", sheet_name="male_dl").replace('---', np.nan).assign(gender='male').assign(event='deadlift')
male_spt = pd.read_excel("scoreChartsACFT.xlsx", sheet_name="male_spt").replace('---', np.nan).assign(gender='male').assign(event='standing power throw')
male_hrp = pd.read_excel("scoreChartsACFT.xlsx", sheet_name="male_hrp").replace('---', np.nan).assign(gender='male').assign(event='hand release pushups')
male_sdc = pd.read_excel("scoreChartsACFT.xlsx", sheet_name="male_sdc").replace('---', np.nan).assign(gender='male').assign(event='sprint drag carry')
male_plk = pd.read_excel("scoreChartsACFT.xlsx", sheet_name="male_plk").replace('---', np.nan).assign(gender='male').assign(event='plank')
male_tmr = pd.read_excel("scoreChartsACFT.xlsx", sheet_name="male_tmr").replace('---', np.nan).assign(gender='male').assign(event='two mile run')

# female scoring charts
female_dl = pd.read_excel("scoreChartsACFT.xlsx", sheet_name="female_dl").replace('---', np.nan).assign(gender='female').assign(event='deadlift')
female_spt = pd.read_excel("scoreChartsACFT.xlsx", sheet_name="female_spt").replace('---', np.nan).assign(gender='female').assign(event='standing power throw')
female_hrp = pd.read_excel("scoreChartsACFT.xlsx", sheet_name="female_hrp").replace('---', np.nan).assign(gender='female').assign(event='hand release pushups')
female_sdc = pd.read_excel("scoreChartsACFT.xlsx", sheet_name="female_sdc").replace('---', np.nan).assign(gender='female').assign(event='sprint drag carry')
female_plk = pd.read_excel("scoreChartsACFT.xlsx", sheet_name="female_plk").replace('---', np.nan).assign(gender='female').assign(event='plank')
female_tmr = pd.read_excel("scoreChartsACFT.xlsx", sheet_name="female_tmr").replace('---', np.nan).assign(gender='female').assign(event='two mile run')

# male alternative cardio
male_walk = pd.read_excel("scoreChartsACFT.xlsx", sheet_name="male_walk").assign(gender='male').assign(event='walk')
male_swim = pd.read_excel("scoreChartsACFT.xlsx", sheet_name="male_swim").assign(gender='male').assign(event='swim')
male_bike = pd.read_excel("scoreChartsACFT.xlsx", sheet_name="male_bike").assign(gender='male').assign(event='bike')
male_row = pd.read_excel("scoreChartsACFT.xlsx", sheet_name="male_row").assign(gender='male').assign(event='row')

# female alternative cardio
female_walk = pd.read_excel("scoreChartsACFT.xlsx", sheet_name="female_walk").assign(gender='female').assign(event='walk')
female_swim = pd.read_excel("scoreChartsACFT.xlsx", sheet_name="female_swim").assign(gender='female').assign(event='swim')
female_bike = pd.read_excel("scoreChartsACFT.xlsx", sheet_name="female_bike").assign(gender='female').assign(event='bike')
female_row = pd.read_excel("scoreChartsACFT.xlsx", sheet_name="female_row").assign(gender='female').assign(event='row')


sheets = [ # list of all data frames 
    # core six events 
    male_dl, female_dl, male_spt, female_spt, 
    male_hrp, female_hrp, male_sdc, female_sdc, 
    male_plk, female_plk, male_tmr, female_tmr,
    # alternative cardio events... worry about them later 
    male_walk, male_swim, male_bike, male_row,
    female_walk, female_swim, female_bike, female_row
]

In [75]:
female_hrp

,17,22,27,32,37,42,47,52,57,62,Points,gender,event
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,female,hand release pushups
1,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,0,female,hand release pushups
2,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,10,female,hand release pushups
3,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,20,female,hand release pushups
4,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,30,female,hand release pushups
5,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,40,female,hand release pushups
6,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,50,female,hand release pushups
7,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,60,female,hand release pushups
8,11.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,61,female,hand release pushups
9,12.0,11.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,62,female,hand release pushups


In [19]:
# this is just temporary and the better funciton will be developed on it's own py file 

def get_acft_score(soldier_data): 
    """Expects a dictionary of information to calculate the ACFT Score. For now it's only a proof of concept for deadlift scores. 
    male_deadlift is temporarily a global variable. Upon realization that Swift uses FP and OOP, I'll might use an object over dictionary"""
    age_range = [62, 57, 52, 47, 42, 37, 32, 27, 22, 17] # decending min age for each age scale, constant 
   
   # determine which age scale to use 
    if soldier_data['age'] >= 62: 
        age_scale = 62
    else: 
        i = 0 
        while soldier_data['age'] < age_range[i]: 
            i += 1
            age_scale = age_range[i]

    # creating subsets of points and events..maybe a more numpy way of doing this, but this works
    dl_weights = male_dl[age_scale].tolist()
    points = male_dl['Points'].tolist()

    # series of simple for loops to match the individual soldier's performance 
    # on each event with correct points 
    for j in range(len(dl_weights)): 
        if dl_weights[j] != 'nan' and soldier_data['deadlift'] >= dl_weights[j]: 
            dl_index = j
    
    

    return points[dl_index]
            

# example dictionary 

my_score = {
    'age': 30, 
    'gender': 'male', 
    'deadlift' : 320
}

get_acft_score(my_score)



91

### filtering data frames and creating a JSON file 

#### exploratory data analysis 

In [20]:
female_plk

,17,22,27,32,37,42,47,55,57,62,Points
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,100.0,55.0,50.0,45.0,40.0,40.0,40.0,40.0,40.0,40.0,0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,101.0,56.0,51.0,46.0,41.0,41.0,41.0,41.0,41.0,41.0,2
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
...,...,...,...,...,...,...,...,...,...,...,...
97,327.0,322.0,317.0,312.0,307.0,307.0,307.0,307.0,307.0,307.0,96
98,330.0,325.0,320.0,315.0,310.0,310.0,310.0,310.0,310.0,310.0,97
99,334.0,329.0,324.0,319.0,314.0,314.0,314.0,314.0,314.0,314.0,98
100,337.0,332.0,327.0,322.0,317.0,317.0,317.0,317.0,317.0,317.0,99


In [27]:
# probably worth keeping a custom points vector with each data frame, 
# as the number of rows vary 
for df in sheets: 
    print(len(df))

48
48
102
102
48
48
102
102
102
102
102
102
2
2
2
2
2
2
2
2


In [36]:
male_tmr

,17,22,27,32,37,42,47,55,57,62,Points
0,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,0
1,2400,2400,2400,2400,2411,2432,2455,2520,2536,2536,0
2,2358,2358,2358,2358,2409,2430,2453,2518,2534,2534,1
3,2356,2356,2356,2356,2407,2428,2451,2516,2532,2532,2
4,2354,2354,2354,2354,2405,2426,2449,2514,2530,2530,3
...,...,...,...,...,...,...,...,...,...,...,...
97,1428,1438,1445,1454,1506,1520,1548,1628,1658,1658,96
98,1415,1425,1431,1440,1453,1504,1532,1614,1644,1644,97
99,1400,1408,1415,1425,1437,1448,1514,1555,1622,1622,98
100,1342,1350,1358,1406,1420,1429,1454,1534,1555,1555,99


In [77]:
# ideally we can just have a dictionary/JSON object where Points are pre-mapped to the correct float value. 
# this also should eliminate the need for the NaNs 

# proof of concept with female deadlift scores 

ages = female_dl.columns.tolist()
ages.pop()
ages # constant for all data frames, the ages categories don't change for gender or event 

# making a list of dictionaries from all of the data frames 
# goal: have something that's easy to subset and format 
# in a way that is efficient and convenient to search
points = female_dl['Points'].tolist() # going to need to be nested into a loop 

# print(len(points))
female_dl_pt_lbs_dicts = [] # before moving on to all the data frames... let's eliminate the nans 
for age in ages: 
    age_scale = female_dl[age].tolist()
    # print(len(age_scale))
    # print(age_scale)
    for i in range(len(age_scale)): 
        if isinstance(age_scale[i], float) and not pd.isna(age_scale[i]): 
            female_dl_pt_lbs_dicts.append({'age': age,'raw': age_scale[i], 'points': points[i]})


# triple for loop...maybe not the most efficient, but it'll buff 
# output will be a list of dictionaries of 'age', 'raw' (raw score means lbs lifted, push ups completed, etc.), and 'points'
# will further be subsetted by gender, but that's for later 


ages = female_dl.columns.tolist()
ages.pop()
ages # [17, 22, 27, 32, 37, 42, 47, 55, 57, 62], constant for all events 
# 

age_raw_pts_dicts = []

for sheet in sheets: 
    points = sheet['Points'].tolist() # points vary upon event 
    gender = sheet['gender'].tolist()
    event = sheet['event'].tolist()
    for age in ages: 
        age_scale = sheet[age].tolist()  # makes lists of each age scale by event, e.g. deadlift raw scores for 27+ year olds 
    # print(len(age_scale))
    # print(age_scale)
        for i in range(len(age_scale)): 
            if isinstance(age_scale[i], float) and not pd.isna(age_scale[i]): 
                age_raw_pts_dicts.append({'age': age,'raw': age_scale[i], 'points': points[i], 'gender': gender[i], 'event': event[i]})


age_raw_pts_dicts


[{'age': '17', 'raw': 0.0, 'points': 0, 'gender': 'male', 'event': 'deadlift'},
 {'age': '17',
  'raw': 80.0,
  'points': 0,
  'gender': 'male',
  'event': 'deadlift'},
 {'age': '17',
  'raw': 90.0,
  'points': 10,
  'gender': 'male',
  'event': 'deadlift'},
 {'age': '17',
  'raw': 100.0,
  'points': 20,
  'gender': 'male',
  'event': 'deadlift'},
 {'age': '17',
  'raw': 110.0,
  'points': 30,
  'gender': 'male',
  'event': 'deadlift'},
 {'age': '17',
  'raw': 120.0,
  'points': 40,
  'gender': 'male',
  'event': 'deadlift'},
 {'age': '17',
  'raw': 130.0,
  'points': 50,
  'gender': 'male',
  'event': 'deadlift'},
 {'age': '17',
  'raw': 140.0,
  'points': 60,
  'gender': 'male',
  'event': 'deadlift'},
 {'age': '17',
  'raw': 150.0,
  'points': 61,
  'gender': 'male',
  'event': 'deadlift'},
 {'age': '17',
  'raw': 160.0,
  'points': 64,
  'gender': 'male',
  'event': 'deadlift'},
 {'age': '17',
  'raw': 170.0,
  'points': 67,
  'gender': 'male',
  'event': 'deadlift'},
 {'age': '17'